<a href="https://colab.research.google.com/github/hsym-uec/myWeb_github/blob/main/Simmple_Stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pandas_datareader import data
import datetime as dt
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import matplotlib.style as style



In [4]:
from pandas_datareader import data
import datetime as dt
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import matplotlib.style as style
# import mplfinance as mpf

#日付取得
end_date = dt.date.today()
start_date = end_date + dt.timedelta(weeks=-52)

code = ['9613.JP','9984.JP'] #ntt_data, softbank
name = ['NTT_Data','SoftBank']
# df_ntt = data.DataReader(code, 'stooq', start, end)

#買った時の１株あたりの単価
price = [0,0]
#保有株数
stock = [0,0]

#取得価格＝保有数量x取得単価
price_buy1 = price[0] * stock[0]
price_buy2 = price[1] * stock[1]
price_buy_sum = price_buy1 + price_buy2 #合計

#各株銘柄取得＆終値取得する関数
def get_kb_Close(code, start_date, end_date):
    df = data.DataReader(code,'stooq', start_date, end_date)
    kb_Close = df['Close']
    return kb_Close

DF = pd.DataFrame()

for i in code:
    DF = pd.concat([DF, get_kb_Close(i, start_date, end_date)], axis=1)
DF.columns = name

DF.index = pd.to_datetime(DF.index) #日付データに変換
print(DF)




            NTT_Data  SoftBank
2023-05-25   1989.00   5065.00
2023-05-24   2017.00   5058.00
2023-05-23   2022.00   5180.00
2023-05-22   1983.00   5218.00
2023-05-19   1957.00   5179.00
...              ...       ...
2022-06-03   2028.48   5393.86
2022-06-02   1985.72   5278.38
2022-06-01   2018.53   5276.39
2022-05-31   2012.57   5355.04
2022-05-30   2025.49   5334.13

[244 rows x 2 columns]


In [34]:
import mysql.connector
from mysql.connector import Error
import pandas as pd
from pandas_datareader import data
import datetime as dt
import numpy as np
import time
import matplotlib.pyplot as plt
import matplotlib.style as style

#各株銘柄取得＆最新(前日)の終値取得する関数
def get_kb_Close(code):
    end_date = dt.date.today()
    start_date = end_date + dt.timedelta(days=-3)
    stock_code = str(code) + '.JP'
    df = data.DataReader(stock_code,'stooq', start_date, end_date)
    kb_Close = df['Close']
    return kb_Close[0]

#get_kb_Close(9613)

In [81]:
get_kb_Close(9613)

2001

In [36]:
def create_server_connection(host_name, user_name, password):
    connection = None
    
    try:
        connection = mysql.connector.connect(
            host = host_name,# ホスト名(IPアドレス）
            user = user_name,
            password = password
		)
        print("MySQL Database connection success")
    except Error as err:
        print(f"Error:'{err}")
    return	connection

In [66]:
#買った時の価格(1株)を取得する関数
def buy_kb(code, buy_days):
    #表記：'%year-%month-%day
    startdate = buy_days
    enddate = buy_days
    stock_code = str(code) + '.JP'
    df = data.DataReader(stock_code,'stooq', startdate, enddate)
    if df.empty:
        return None
    buy_kb_Close = df['Close']
    return buy_kb_Close[0]

buy_kb(9613,'2023-05-26')

1982

In [38]:
#取引報告計算(買った場合)
def cal_kb(code, code_name, purchase_num, buy_days):
    connection = None
    passwordy = '507560yosshi'
    
    price = float(buy_kb(code, buy_days)) #買った時の一株あたりの単価
    print(price)
    sum_price = float(price * purchase_num)
    print(sum_price)
    info = 'buy' #売買の判断

    try:
        connection = mysql.connector.connect(
            host = 'localhost',
            user = 'yosshy',
            password = passwordy,
            database = 'stocks_situation'
        )

        cursor = connection.cursor()

        sql = ('''
        insert into transactions
            (stock_code, company_name, trade_situation, trade_day, amount, trade_price, sum_trade_price) 
            values (%s, %s, %s, %s, %s, %s, %s)
        ''')

        val = [
            (code, code_name, info, buy_days, purchase_num, price, sum_price)
        ]
        print(val)

        cursor.executemany(sql, val)
        connection.commit()
        
        print(f"MySQL INSERT success")

        cursor.close()
    except Error as err:
        print(f"Error:' {err}")
    
    finally:
        if connection is not None and connection.is_connected():
            connection.close()

# cal_kb(9613, 'NTT_Data', 100, '2023-05-25')


	

In [6]:
cal_kb(6758, 'Sony', 100, '2023-05-18')

13720.0
1372000.0
MySQL INSERT success


In [40]:
cal_kb(7974, 'Nintendo', 100, '2023-05-29')

5909.0
590900.0
[(7974, 'Nintendo', 'buy', '2023-05-29', 100, 5909.0, 590900.0)]
MySQL INSERT success


In [82]:
#保有株式の状況
def now_kb():
    connection = None
    passwordy = '507560yosshi'

    count = 0
    code_list = []
    company_list = []
    amount_list = []


    try:
        connection = mysql.connector.connect(
            host = 'localhost',
            user = 'yosshy',
            password = passwordy,
            database = 'stocks_situation'
        )

        cursor = connection.cursor(dictionary=True)

        #trade_situationの判定をしない場合(仮)
        get_sql = ('''
            select id, stock_code, company_name, amount from transactions;
        ''')

        insert_sql = ('''
            insert into now_stocks
            (stock_code, company_name, now_price, amount, market_price, now_date)
            values (%s, %s, %s, %s, %s, %s)
        ''')

        cursor.execute(get_sql)
        
        for fetched_line in cursor.fetchall():
            id = fetched_line['id']
            # print(id)
            stock_code = fetched_line['stock_code']
            company_name = fetched_line['company_name']
            amount = fetched_line['amount']
            #保有株式のコードのリスト
            code_list.insert((id-1), stock_code)
            #保有株式の名前のリスト
            company_list.insert((id-1), company_name)
            #保有株式の保有数のリスト
            amount_list.insert((id-1), amount)
            count += 1

        # print(code_list)
        for i in range(count):
            code = int(code_list[i])
            code_name = company_list[i]
            amounts = int(amount_list[i])
            if code is None or amounts is None:
                continue
            now_price = float(get_kb_Close(code))
            # now_price = float(buy_kb(code, '2023-05-26'))
            print(now_price)
            market_price = float(amounts * now_price)
            # now_date = dt.date.today() + dt.timedelta(days=-5)
            now_date = dt.date.today() + dt.timedelta(days=-1)
            val = [(code, code_name, now_price, amounts, market_price, now_date)]
            print(f"{i+1}: MySQL INSERT success")
            print(val)
            cursor.executemany(insert_sql, val)
            connection.commit()
            

        cursor.close()

    except Error as err:
        print(f"Error:' {err}")
    
    finally:
        if connection is not None and connection.is_connected():
            connection.close()

In [83]:
now_kb()

2001.0
1: MySQL INSERT success
[(9613, 'NTT_Data', 2001.0, 100, 200100.0, datetime.date(2023, 5, 30))]
13275.0
2: MySQL INSERT success
[(6758, 'Sony', 13275.0, 100, 1327500.0, datetime.date(2023, 5, 30))]
5909.0
3: MySQL INSERT success
[(7974, 'Nintendo', 5909.0, 100, 590900.0, datetime.date(2023, 5, 30))]


In [80]:
now_date = dt.date.today()
print(now_date)

2023-05-31


In [75]:
now_price = float(get_kb_Close(9613))
print(now_price)

2001.0
